In [1]:
#a simple DL architecture to classify MNIST digits
# Load the TensorBoard notebook extension
%load_ext tensorboard


In [2]:
import tensorflow as tf
import datetime

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

11490434/11490434 [==============================] - 0s 0us/step


In [8]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2202 - accuracy: 0.9359 - val_loss: 0.1101 - val_accuracy: 0.9674
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0980 - accuracy: 0.9703 - val_loss: 0.0811 - val_accuracy: 0.9744
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0690 - accuracy: 0.9779 - val_loss: 0.0804 - val_accuracy: 0.9746
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0540 - accuracy: 0.9824 - val_loss: 0.0708 - val_accuracy: 0.9762
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0424 - accuracy: 0.9865 - val_loss: 0.0615 - val_accuracy: 0.9821


In [11]:
%tensorboard --logdir logs/fit


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [13]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [14]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')



In [15]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [16]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [17]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24161206185817719, Accuracy: 92.98833465576172, Test Loss: 0.11529594659805298, Test Accuracy: 96.5999984741211
Epoch 2, Loss: 0.10251345485448837, Accuracy: 96.93833923339844, Test Loss: 0.07867030054330826, Test Accuracy: 97.58999633789062
Epoch 3, Loss: 0.0741565003991127, Accuracy: 97.71833801269531, Test Loss: 0.07117687165737152, Test Accuracy: 97.81999969482422
Epoch 4, Loss: 0.05342089384794235, Accuracy: 98.29999542236328, Test Loss: 0.06545010954141617, Test Accuracy: 97.91999816894531
Epoch 5, Loss: 0.04420112073421478, Accuracy: 98.63333129882812, Test Loss: 0.06398884207010269, Test Accuracy: 98.1199951171875


In [18]:
%tensorboard --logdir logs/gradient_tape

